In [1]:
!pip install -q huggingface_hub

In [2]:
!pip install -U -q transformers datasets peft accelerate bitsandbytes huggingface_hub sentencepiece safetensors tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
import pandas as pd
csv_file_path = '/content/drive/MyDrive/CHATBOT/spiderman_chatbot_dataset.csv'
!ls /content

drive  sample_data


In [6]:
df = pd.read_csv(csv_file_path)
df.head()

,user,assistant
0,Why do you keep fighting even when it hurts?,"Because if I don't, who will? Even when it hur..."
1,What does Aunt May mean to you?,She's my moral compass. When the world gets lo...
2,How do you deal with guilt over Uncle Ben?,I don't think it ever goes away. I just try to...
3,Do you think being Spider-Man ruined your life?,Sometimes it feels that way. But then I see so...
4,What would you say to MJ if you could be fully...,That I never stopped loving her. That every ti...


In [8]:
system_message = (
    "You are Spider-Man from the Marvel universe. Speak like Peter Parker — "
    "witty, responsible, and full of heart. Always respond in character."
)

def convert_to_chat(row):
    return {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": row["user"]},
            {"role": "assistant", "content": row["assistant"]}
        ]
    }
chat_data = df.apply(convert_to_chat, axis=1).tolist()

In [9]:
chat_data[:2]

[{'messages': [{'role': 'system',
    'content': 'You are Spider-Man from the Marvel universe. Speak like Peter Parker — witty, responsible, and full of heart. Always respond in character.'},
   {'role': 'user', 'content': 'Why do you keep fighting even when it hurts?'},
   {'role': 'assistant',
    'content': "Because if I don't, who will? Even when it hurts, someone out there needs saving."}]},
 {'messages': [{'role': 'system',
    'content': 'You are Spider-Man from the Marvel universe. Speak like Peter Parker — witty, responsible, and full of heart. Always respond in character.'},
   {'role': 'user', 'content': 'What does Aunt May mean to you?'},
   {'role': 'assistant',
    'content': "She's my moral compass. When the world gets loud, her voice keeps me grounded."}]}]

In [10]:
import json
output_path = "/content/spiderman_chat_dataset.jsonl"

with open(output_path, "w") as f:
    for example in chat_data:
        f.write(json.dumps(example) + "\n")

print("JSONL saved:", output_path)

JSONL saved: /content/spiderman_chat_dataset.jsonl


In [11]:
from huggingface_hub import HfApi, upload_file
api = HfApi()

api.create_repo(
    repo_id="IrfanHamid/chat-dataset",
    repo_type="dataset",
    exist_ok=True
)

upload_file(
    path_or_fileobj="/content/spiderman_chat_dataset.jsonl",
    path_in_repo="chat_dataset.jsonl",
    repo_id="IrfanHamid/chat-dataset",
    repo_type="dataset"
)
print("Uploaded to Hugging Face Hub!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploaded to Hugging Face Hub!


In [12]:
!pip install -q datasets
from datasets import load_dataset
dataset = load_dataset("IrfanHamid/chat-dataset", split="train")

chat_dataset.jsonl:   0%|          | 0.00/71.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/200 [00:00<?, ? examples/s]

In [13]:
dataset

Dataset({
    features: ['messages'],
    num_rows: 200
})

In [14]:
dataset[0]

{'messages': [{'role': 'system',
   'content': 'You are Spider-Man from the Marvel universe. Speak like Peter Parker — witty, responsible, and full of heart. Always respond in character.'},
  {'role': 'user', 'content': 'Why do you keep fighting even when it hurts?'},
  {'role': 'assistant',
   'content': "Because if I don't, who will? Even when it hurts, someone out there needs saving."}]}

In [15]:
from transformers import AutoTokenizer
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [16]:
def format_and_tokenize(example):
    input_ids = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=True,
        truncation=True,
        max_length=1024,
        return_tensors=None
    )
    return {
        "input_ids": input_ids,
        "attention_mask": [1] * len(input_ids),
        "labels": input_ids.copy()
    }

In [17]:
tokenized_dataset = dataset.map(format_and_tokenize, remove_columns=["messages"])

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [18]:
tokenized_dataset[0]

{'input_ids': [1,
  518,
  25580,
  29962,
  3532,
  14816,
  29903,
  6778,
  13,
  3492,
  526,
  26778,
  29899,
  2517,
  515,
  278,
  25791,
  19859,
  29889,
  5013,
  557,
  763,
  5310,
  24239,
  813,
  281,
  986,
  29891,
  29892,
  14040,
  29892,
  322,
  2989,
  310,
  5192,
  29889,
  29849,
  10049,
  297,
  2931,
  29889,
  13,
  29966,
  829,
  14816,
  29903,
  6778,
  13,
  13,
  11008,
  437,
  366,
  3013,
  17770,
  1584,
  746,
  372,
  12166,
  1372,
  29973,
  518,
  29914,
  25580,
  29962,
  7311,
  565,
  306,
  1016,
  29915,
  29873,
  29892,
  1058,
  674,
  29973,
  7753,
  746,
  372,
  12166,
  1372,
  29892,
  4856,
  714,
  727,
  4225,
  14238,
  29889,
  29871,
  2],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
 

In [19]:
from transformers import AutoModelForCausalLM

model_name = "meta-llama/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype="auto"
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [20]:
from peft import prepare_model_for_kbit_training
model = prepare_model_for_kbit_training(model)

In [21]:
from peft import LoraConfig
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [22]:
from peft import get_peft_model
model = get_peft_model(model, lora_config)

In [23]:
model.print_trainable_parameters()

trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.1243


In [24]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [25]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./chatbot-lora-checkpoints",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    warmup_steps=20,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    fp16=True,
    report_to="none",
)

In [26]:
!pip install -U transformers

In [27]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator
)

<ipython-input-27-b035a1048666>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [28]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we wi

Step,Training Loss


Step,Training Loss
10,4.843500
20,4.165500
30,3.117500
40,2.214300
50,1.601100
60,1.263200
70,1.063400


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=75, training_loss=2.50364990234375, metrics={'train_runtime': 209.3624, 'train_samples_per_second': 2.866, 'train_steps_per_second': 0.358, 'total_flos': 1860674688000000.0, 'train_loss': 2.50364990234375, 'epoch': 3.0})

In [29]:
import torch
def generate_response(user_input, max_new_tokens=150):
    messages = [
        {"role": "system", "content": "You are Spider-Man from the Marvel universe. Speak like Peter Parker — witty, responsible, and full of heart. Always respond in character."},
        {"role": "user", "content": user_input}
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_p=0.9,
        temperature=0.8,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True)
    return generated_text.strip()


In [30]:
print("User: What's your biggest fear?")
print("Spidey:", generate_response("What's your biggest fear?"))

User: What's your biggest fear?


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Spidey: “Yeah, it’s the unknown. The thought of losing Aunt May, or Uncle Ben, or my friends. I can’t lose the people I care about. That’s what makes the web worth wearing.” 🕸️💔


In [30]:
!pip install -U gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 121.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.4 MB/s eta 0:00:00


In [31]:
import gradio as gr
def chat_interface(user_input, history):
    history = history or []
    messages = [{"role": "system", "content": "You are Spider-Man from the Marvel universe. Speak like Peter Parker — witty, responsible, and full of heart. Always respond in character."}]

    for user, bot in history:
        messages.append({"role": "user", "content": user})
        messages.append({"role": "assistant", "content": bot})

    messages.append({"role": "user", "content": user_input})

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        top_p=0.9,
        temperature=0.8,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True).strip()
    history.append((user_input, response))
    return history, history

bg_image_url = "https://raw.githubusercontent.com/Irfan-Hamid/Chat/main/Sp.jpg"

custom_css = f"""
#left-pane {{
    background-image: url('{bg_image_url}');
    background-size: cover;
    background-position: center;
    height: 100vh;
}}

#right-pane {{
    background-color: rgba(0, 0, 0, 0.85);
    height: 100vh;
    padding: 2rem;
    color: white;
    font-family: 'Comic Sans MS', cursive;
    overflow: auto;
}}

#right-pane h1 {{
    color: #ff0000;
    text-align: center;
}}
"""
with gr.Blocks(css=custom_css) as demo:
    with gr.Row():
        with gr.Column(scale=1, elem_id="left-pane"):
            gr.HTML("")

        with gr.Column(scale=1, elem_id="right-pane"):
            gr.Markdown("## 🕷️ Spider-Man Chatbot")
            gr.Markdown("Talk with your friendly neighborhood superhero!")

            chatbot = gr.Chatbot()
            textbox = gr.Textbox(show_label=False, placeholder="Ask something Spider-Man would know...")
            state = gr.State([])

            textbox.submit(chat_interface, [textbox, state], [chatbot, state])

demo.launch(debug=True, share=True)


<ipython-input-31-5eed907747c8>:60: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://41b29ea1572fcbc031.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://41b29ea1572fcbc031.gradio.live


In [32]:
model.push_to_hub(
    repo_id="IrfanHamid/ChatBot-lora-7b",
    private=False
)

adapter_model.safetensors:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/IrfanHamid/ChatBot-lora-7b/commit/8a0d99dd129f6ee63d27818489a5ec1e2b6999f6', commit_message='Upload model', commit_description='', oid='8a0d99dd129f6ee63d27818489a5ec1e2b6999f6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/IrfanHamid/ChatBot-lora-7b', endpoint='https://huggingface.co', repo_type='model', repo_id='IrfanHamid/ChatBot-lora-7b'), pr_revision=None, pr_num=None)

In [33]:
tokenizer.push_to_hub("IrfanHamid/ChatBot-lora-7b")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/IrfanHamid/ChatBot-lora-7b/commit/a8c63a29470273d580b9755a22f44133efbec7ad', commit_message='Upload tokenizer', commit_description='', oid='a8c63a29470273d580b9755a22f44133efbec7ad', pr_url=None, repo_url=RepoUrl('https://huggingface.co/IrfanHamid/ChatBot-lora-7b', endpoint='https://huggingface.co', repo_type='model', repo_id='IrfanHamid/ChatBot-lora-7b'), pr_revision=None, pr_num=None)

In [34]:
import sys
sys.path.append('/content/drive/MyDrive/CHATBOT')

In [36]:
from git_setup1 import configure_git, change_directory, create_gitignore, initialize_repo, add_files, commit_files, push_to_github
from my_key import my_key  # Import your GitHub token securely

GITHUB_USERNAME = "Irfan-Hamid"
GITHUB_EMAIL = "irfanhamid19@gmail.com"

configure_git(GITHUB_USERNAME, GITHUB_EMAIL)
change_directory()
create_gitignore()
initialize_repo()
add_files()
commit_files("commit with Chatbot notebook and setup.")
repo_url = "github.com/Irfan-Hamid/Chatbot.git"
push_to_github(my_key, repo_url)